# Análisis postgrados posterior al MAV 

In [1]:
import socket
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import _plotly_utils.basevalidators
import plotly.express as px
from plotly.figure_factory import create_table
import plotly.figure_factory as ff
import plotly.offline as po
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from bokeh.layouts import column, row
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import Figure, show, output_notebook
output_notebook()

Loading BokehJS ...

## Tabla de Postgrados

In [2]:
path_file    = 'egresados_mag_doc.xlsx'
df_egresados = pd.read_excel(path_file, sheet_name='egresados', index_col=0)
fields       = ['mail', 'Programa_Mag_Doc', 'Nombre_programa', 'Fuente_recursos', 'Institucion_Mag_Doc', 
                'Pais_mag_doc', 'Rank_QS', 'Pregunta_cap_mav']
df_egresados_subset_columns = pd.read_excel(path_file, index_col=0, usecols=fields)

In [3]:
large_rockwell_template = dict(
    layout=go.Layout(title_font=dict(family="Rockwell", size=16))
)

fig = go.Figure(data=[go.Table(
    header=dict(values=list('<b>' + df_egresados_subset_columns.columns + '</b>'),
    fill_color='darkblue', 
    align='left', font=dict(
        family="Rockwell", color='white', size=12), height=20),
    cells=dict(values=[df_egresados_subset_columns.Programa_Mag_Doc, 
                       df_egresados_subset_columns.Nombre_programa, 
                       df_egresados_subset_columns.Fuente_recursos, 
                       df_egresados_subset_columns.Institucion_Mag_Doc,
                       df_egresados_subset_columns.Rank_QS,
                       df_egresados_subset_columns.Pais_mag_doc,
                       df_egresados_subset_columns.Pregunta_cap_mav],
    fill_color='lavender',
    align='left', font=dict(
        family="Rockwell", size=12), height=30))
    ])
fig.update_layout(width=1000, height=800)    

fig.update_layout(title="Tabla. Posterior al MAV, continuaron otro postgrado encuestados Graduados 2012-2021",
                  template=large_rockwell_template)

fig.show()

## ¿Cuántos continuaron a otros postgrados?

In [4]:
temp_series = df_egresados['Programa_Mag_Doc'].value_counts()
labels = (np.array(temp_series.index))
sizes = (np.array((temp_series / temp_series.sum())*100))

trace = go.Pie(labels=labels, values=sizes, marker=dict(line=dict(color='#797D7F', width=1.5)), hole=.4)
layout = go.Layout(
    title='Posterior al MAV continuaron otro postgrado encuestados Graduados 2012-2021',
    title_font_family="Rockwell", 
    annotations=[dict(text='Continuaron postgrado', x=0.5, y=0.5, font_size=14, showarrow=False)],
    font=dict(
        family="Rockwell", size=14)
)
data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="gender")

## ¿En qué países siguieron otros estudios de postgrado?

In [5]:
path_file_sorted    = 'egresados_sorted.xlsx'
df_egresados_sorted = pd.read_excel(path_file_sorted, sheet_name='egresados', index_col=0)

loc_df = pd.read_csv("location_map.csv")
new_df = pd.merge(df_egresados_subset_columns[["Pais_mag_doc","Programa_Mag_Doc"]], loc_df, left_on="Pais_mag_doc", right_on="COUNTRY")
new_df = new_df.groupby(["Pais_mag_doc", "CODE"])["Programa_Mag_Doc"].aggregate('count').reset_index()
new_df.columns = ["Pais_mag_doc", "Code", "Count"]

data = [ dict(
        type = 'choropleth',
        locations = new_df['Code'],
        z = new_df['Count'],
        text = new_df['Pais_mag_doc'],
        colorscale = [[0,"rgb(210, 210, 210)"], [4500,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            title = 'Cuenta'),
      ) ]

layout = dict(
    title = 'Países en los que continuaron postgrados encuestados Graduados 2012-2021', font=dict(
        family="Rockwell", size=12),
    geo = dict(
        showframe = False,
        showcoastlines = True,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-world-map' )

## ¿Cuáles fueron las fuentes de financiamiento para continuar postgrados?

In [6]:
temp_series = df_egresados['Fuente_recursos'].value_counts()
labels = (np.array(temp_series.index))
sizes = (np.array((temp_series / temp_series.sum())*100))

trace = go.Pie(labels=labels, values=sizes, marker=dict(line=dict(color='#797D7F', width=1.5)), hole=.4)
layout = go.Layout(
    title='Distribución de fuentes de financiamiento para seguir otros postgrados',
    title_font_family="Rockwell", 
    annotations=[dict(text='Financiamiento', x=0.5, y=0.5, font_size=12, showarrow=False)],
    font=dict(
        family="Rockwell", size=12)
)
data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="gender")

## ¿Cuáles son las Universidades y áreas de postgrados?

In [7]:
x =['Pontificia Universidad Católica de Chile\n, Ingeniería Industrial', 
       'Universidad de Chile, Ingeniería Eléctrica', 
       'The University of Auckland, Ingeniería Eléctrica y Computación',
       'Université de Genève, Física',
       'Universidad del Valle, Ingeniería Mecánica',
       'Universidad Politécnica de Madrid, Ingeniería Mecánica',
       'Universidade Estadual de Campinas, Matemática',
       'Universidad Politécnica de Valencia, Telecomunicación']
y =[135, 
       183, 
       85, 
       105,
       801, 
       459,
       219,
       371]

texts = ["135","183","85","105","801", "459", '219', '371']
positions = ['inside','inside','inside','inside','inside', 'inside','inside','inside']
fig = px.bar(x=x, y=y, color=df_egresados_subset_columns.Rank_QS, template='ggplot2')
fig.update_layout(
    title="Países de origen de encuestados Graduados 2012-2021", yaxis_title='Ranking QS Universidades', font=dict(
        family="Rockwell", size=14),
    xaxis_title="Universidad y Programa de postgrado")
fig.update_traces(texttemplate = texts, textposition = positions)
fig.show()

## Pregunta: ¿En el programa de postgrado que está realizando (o realizó), ¿en qué medida aprovecha (o aprovechó) las capacidades adquiridas en su programa?

In [8]:
path_file_sorted    = 'egresados_sorted.xlsx'
df_egresados_sorted = pd.read_excel(path_file_sorted, sheet_name='egresados', index_col=0)

In [9]:
cols_to_use  = ["Pregunta_cap_mav"]
df_egresados_sorted.Pregunta_cap_mav
df_egresados_sorted.rename(columns = {'Pregunta_cap_mav' : '¿En qué medida aprovecha (o aprovechó) las capacidades adquiridas en su programa? '}, inplace = True)

In [10]:
names           = []
nunca_count     = []
rarely_count    = []
often_count     = []
sometimes_count = []
mostly_count    = []

var_col_name   = '¿En qué medida aprovecha (o aprovechó) las capacidades adquiridas en su programa? '
title_col_name = '¿En qué medida aprovecha (o aprovechó) <br> las capacidades adquiridas en su programa? '
names.append(title_col_name)

cols_to_use  = [var_col_name]
for col in cols_to_use:
    cnt_srs = df_egresados_sorted[col].value_counts()
    sometimes_count.append(cnt_srs["Con frecuencia"])
    mostly_count.append(cnt_srs["Siempre"])

trace0 = go.Bar(
    y=names,
    x=nunca_count,
    orientation = 'h',
    name = "nunca"
)    
trace1 = go.Bar(
    y=names,
    x=rarely_count,
    orientation = 'h',
    name = "Raras veces"
)
trace2 = go.Bar(
    y=names,
    x=sometimes_count,
    orientation = 'h',
    name = "Con frecuencia"
)
trace3 = go.Bar(
    y=names,
    x=often_count,
    orientation = 'h',
    name = "Ocasionalmente"
)
trace4 = go.Bar(
    y=names,
    x=mostly_count,
    orientation = 'h',
    name = "Siempre"
)    

layout = go.Layout(
    title='En el programa de postgrado que está realizando (o realizó)...' ,
    barmode = 'stack',
    width   = 900,
    height  = 250
)

data = [trace0, trace1, trace2, trace3, trace4]
fig = go.Figure(data=data, layout=layout)

fig.update_layout(
    font_family="Rockwell",
    font_color="black",
    font_size=12,
    title_font_family="Rockwell", title_font_size= 14,
    title_font_color="black",
    legend_title_font_color="green"
)

py.iplot(fig, filename="WorkToolsFrequency")